In [1]:
import ISRAnalysis as an

Welcome to JupyROOT 6.14/04


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys

# Unfolding for detector (up to dressed lepton level) 

In [3]:
binDef = "_FineCoarse"
channel = "electron"
year = "2016"
doSys = True
regularisation_mode = 0

# Redirect STDOUT and STDERR
sys.stderr = open("./log/stderr_"+channel+"_"+year+".txt", "w")
sys.stdout = open("./log/stdout_"+channel+"_"+year+".txt", "w")

detUnfold = an.ISRAnalysis("DetUNFOLD", year, channel, regularisation_mode, False, "matrix", 
                                     "Detector_Dressed_DRp1_Fiducial", "Detector_Dressed_DRp1", binDef)


In [4]:
# Nominal
detUnfold.setInputHist(False, False, None, "Detector")
detUnfold.setUnfoldBkgs(False, "Detector")
detUnfold.subFake() # Subract DY fake (out of fiducial volume at gen level)
#detUnfold_muon_2016.subFake(False, "detector_level_DY_Fake") # Subract DY fake (out of fiducial volume at gen level)

In [5]:
if doSys == True:
    if channel == "muon" :
        systematics = {
                       "ID":["IdSFUp", "IdSFDown"], 
                       "ISO":["IsoSFUp", "IsoSFDown"], 
                       "TRG":["trgSFUp", "trgSFDown"], 
                       #"L1Prefie":["L1PrefireUp","L1PrefireDown"], 
                       "PU":["PUUp", "PUDown"],
                       #"#ZptCorr": ["ZpTCorrected", "Nominal"], # use different matrix file
                       #"LepScale": ["LepScaleUp", "LepScaleDown"],
                       "Unfolding": ["unfoldIterEM", "unfoldNominal"],
                       "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], 
                       "AlphaS":["AlphaSUp", "AlphaSDown"],
                       "PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
                       #"Background": ["NormUp", "NormDown"],
                       }
    
    if channel == "electron" :
        systematics = {
                       "ID":["IdSFUp", "IdSFDown"],
                       "Reco":["recoSFUp","recoSFDown"], 
                       "TRG":["trgSFUp", "trgSFDown"], 
                       "L1Prefie":["L1PrefireUp","L1PrefireDown"], 
                       "PU":["PUUp", "PUDown"],
                       #"ZptCorr": ["ZpTCorrected", "Nominal"],
                       #"LepScale": ["LepScaleUp", "LepScaleDown"],
                       #"LepRes": ["LepResUp", "LepResDown"],
                       "Unfolding": ["unfoldIterEM", "unfoldNominal"],
                       "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], 
                       "AlphaS":["AlphaSUp", "AlphaSDown"],
                       "PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
                       "Background": ["NormUp", "NormDown"],
                        }
    
    for sysName, postfixs in systematics.items():
        for postfix in postfixs:
            detUnfold.setSystematics(sysName, postfix)
            #print(sysName + " " + postfix)
            detUnfold.setInputHist(False, False, None, "Detector", True, sysName, postfix)
            detUnfold.setUnfoldBkgs(True, "Detector", sysName, postfix) #Combine Bkgs and Fake?
            histpostfix = postfix
            if sysName == "Unfolding" or "LepScale" in sysName or "LepRes" in sysName:
                histpostfix = ""
            detUnfold.subFake(True, "Detector_DY_Fake", sysName, postfix, histpostfix)
            #detUnfold_.subFake(True, "detector_level_DY_Fake", sysName, postfix, histpostfix)

In [6]:
#detUnfold.drawResponseM("Pt")
#detUnfold.drawResponseM("Mass")

detUnfold.checkMatrixCond()

<ROOT.TVectorT<double> object ("TVectorT<double>") at 0xb23b5f8>

In [7]:
detUnfold.doUnfold() # nominal unfold
if doSys:
    detUnfold.doUnfold(doSys)

In [8]:
detUnfold.doAcceptance(doSys, False, "DetEfficienyCorr")

# Unfolding for QED FSR

In [9]:
binDef = "_CoarseCoarse"
phase_space = "Fiducial"

fsrUnfold = an.ISRAnalysis("FSRUNFOLD", year, channel, 0, True, "fsr_matrix", 
                                     "Dressed_DRp1_Dressed_DR4PI_"+phase_space, "Dressed_DRp1_Dressed_DR4PI", binDef)

In [10]:
fsrUnfold.setInputHist(False, True, detUnfold) # Set nominal input using previous unfolding result
fsrUnfold.subFake(False, "Dressed_DRp1_DY_Fake", "Nominal", "", "", True)

In [11]:
#fsrUnfold.drawResponseM("Pt")
#fsrUnfold.drawResponseM("Mass")

fsrUnfold.checkMatrixCond()

<ROOT.TVectorT<double> object ("TVectorT<double>") at 0x15507c68>

In [12]:
if doSys :
    fsr_systematics = {
                       "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], "AlphaS":["AlphaSUp", "AlphaSDown"],
                       "PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
                       "Unfolding": ["unfoldIterEM", "unfoldNominal"],
                       "FSR": ["fsrPHOTOS","fsrPYTHIA"],
    }
    
    for sysName, postfixs in fsr_systematics.items():
        for postfix in postfixs:
            fsrUnfold.setSystematics(sysName, postfix, True) # Create TUnfoldDensity objects for systematics
            
            if "FSR" in sysName:
                fsrUnfold.setInputHist(False, True, detUnfold, "", True, sysName, postfix)
                fsrUnfold.subFake(True, "Dressed_DRp1_DY_Fake", sysName, postfix, postfix, True)

In [13]:
fsrUnfold.setFromPreviousUnfold(detUnfold) # Set using previous unfolding result for systematics

In [14]:
if doSys:
    for sysName, postfixs in systematics.items():
        for postfix in postfixs:
            fsrUnfold.subFake(True, "Dressed_DRp1_DY_Fake", sysName, postfix, "", True)

In [15]:
fsrUnfold.doUnfold() # do nominal unfold
if doSys : 
    fsrUnfold.doUnfold(doSys) # do unfold for systematic

In [16]:
fsrUnfold.doStatUnfold() # NOTE: doStatUnfold() must be done after setMeanValues() and before any other processes
                                   # Don't know why exactly...

# Acceptance correction 

In [17]:
fsrUnfold.doAcceptance(doSys, True)